In [1]:
# Importing the libraries needed
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
import seaborn as sns
import transformers
import json
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel
#from transformers.configuration_bert import BertConfig
import logging
logging.basicConfig(level=logging.ERROR)

In [2]:
!pwd

/Users/shashanksahoo/Documents/KIT_Projects_2022/ICA_project_2023/kit-docgcn-dla-service/notebooks


In [4]:
import os
os.chdir('../py-bottom-up-attention')

In [5]:
import os
train_file_list = os.listdir('../data/training_data/annotations')
print(len(train_file_list))
training_list = []
for i in train_file_list:
  path = '../data/training_data/annotations/'+i
  with open(path) as f:
    d = json.load(f)
    for x in d['form']:
      training_list.append(x)

149


In [6]:
import os
test_file_list = os.listdir('../data/testing_data/annotations')
print(len(test_file_list))
test_list = []
for i in test_file_list:
  path = '../data/testing_data/annotations/'+i
  with open(path) as f:
    d = json.load(f)
    for x in d['form']:
      test_list.append(x)

50


In [40]:
training_list[:5]

[{'box': [394, 145, 433, 162],
  'text': 'ITEM:',
  'label': 'question',
  'words': [{'box': [394, 145, 433, 162], 'text': 'ITEM:'}],
  'linking': [[0, 15]],
  'id': 0},
 {'box': [109, 112, 151, 129],
  'text': 'DATE:',
  'label': 'question',
  'words': [{'box': [109, 112, 151, 129], 'text': 'DATE:'}],
  'linking': [[1, 13]],
  'id': 1},
 {'box': [110, 140, 159, 155],
  'text': 'BRAND:',
  'label': 'question',
  'words': [{'box': [110, 140, 159, 155], 'text': 'BRAND:'}],
  'linking': [[2, 14]],
  'id': 2},
 {'box': [109, 183, 168, 198],
  'text': 'SUMMARY',
  'label': 'question',
  'words': [{'box': [109, 183, 168, 198], 'text': 'SUMMARY'}],
  'linking': [],
  'id': 3},
 {'box': [175, 184, 193, 195],
  'text': 'OF',
  'label': 'question',
  'words': [{'box': [175, 184, 193, 195], 'text': 'OF'}],
  'linking': [],
  'id': 4}]

In [8]:
test_list[:5]

[{'box': [103, 268, 131, 282],
  'text': 'Date:',
  'label': 'question',
  'words': [{'box': [103, 268, 131, 282], 'text': 'Date:'}],
  'linking': [[0, 20]],
  'id': 0},
 {'box': [102, 297, 120, 311],
  'text': 'To:',
  'label': 'question',
  'words': [{'box': [102, 297, 120, 311], 'text': 'To:'}],
  'linking': [[1, 13]],
  'id': 1},
 {'box': [103, 328, 159, 345],
  'text': 'Company:',
  'label': 'question',
  'words': [{'box': [103, 328, 159, 345], 'text': 'Company:'}],
  'linking': [[2, 5]],
  'id': 2},
 {'box': [381, 297, 416, 314],
  'text': 'From:',
  'label': 'question',
  'words': [{'box': [381, 297, 416, 314], 'text': 'From:'}],
  'linking': [[3, 16]],
  'id': 3},
 {'box': [563, 360, 574, 373],
  'text': '3',
  'label': 'answer',
  'words': [{'box': [563, 360, 574, 373], 'text': '3'}],
  'linking': [[15, 4]],
  'id': 4}]

In [9]:
from pandas import DataFrame
df_train = DataFrame(training_list)
df_train.head(100)

,box,text,label,words,linking,id
0,"[394, 145, 433, 162]",ITEM:,question,"[{'box': [394, 145, 433, 162], 'text': 'ITEM:'}]","[[0, 15]]",0
1,"[109, 112, 151, 129]",DATE:,question,"[{'box': [109, 112, 151, 129], 'text': 'DATE:'}]","[[1, 13]]",1
2,"[110, 140, 159, 155]",BRAND:,question,"[{'box': [110, 140, 159, 155], 'text': 'BRAND:'}]","[[2, 14]]",2
3,"[109, 183, 168, 198]",SUMMARY,question,"[{'box': [109, 183, 168, 198], 'text': 'SUMMAR...",[],3
4,"[175, 184, 193, 195]",OF,question,"[{'box': [175, 184, 193, 195], 'text': 'OF'}]",[],4
...,...,...,...,...,...,...
95,"[84, 214, 143, 232]",Company,question,"[{'box': [84, 214, 143, 232], 'text': 'Company'}]","[[1, 22]]",1
96,"[601, 397, 640, 412]",DATE,question,"[{'box': [601, 397, 640, 412], 'text': 'DATE'}]","[[2, 13]]",2
97,"[412, 395, 493, 409]",SIGNATURE,question,"[{'box': [412, 395, 493, 409], 'text': 'SIGNAT...","[[3, 34]]",3
98,"[223, 395, 265, 410]",NAME,question,"[{'box': [223, 395, 265, 410], 'text': 'NAME'}]","[[4, 32], [4, 33]]",4


In [10]:
df_train.label.value_counts()

question    3266
answer      2802
other        902
header       441
Name: label, dtype: int64

In [11]:
df_train.text

0                                                ITEM:
1                                                DATE:
2                                               BRAND:
3                                              SUMMARY
4                                                   OF
                             ...                      
7406    Topline w/o 12 /18 /95 Final Report 12/ 31/ 95
7407                                          $100 600
7408                            Revised Costs (if any)
7409                               Fieldwork Schedule:
7410                                    A & G Research
Name: text, Length: 7411, dtype: object

In [12]:
df_train.isna().sum()

box        0
text       0
label      0
words      0
linking    0
id         0
dtype: int64

In [13]:
from pandas import DataFrame
df_test = DataFrame(test_list)
df_test.head()

,box,text,label,words,linking,id
0,"[103, 268, 131, 282]",Date:,question,"[{'box': [103, 268, 131, 282], 'text': 'Date:'}]","[[0, 20]]",0
1,"[102, 297, 120, 311]",To:,question,"[{'box': [102, 297, 120, 311], 'text': 'To:'}]","[[1, 13]]",1
2,"[103, 328, 159, 345]",Company:,question,"[{'box': [103, 328, 159, 345], 'text': 'Compan...","[[2, 5]]",2
3,"[381, 297, 416, 314]",From:,question,"[{'box': [381, 297, 416, 314], 'text': 'From:'}]","[[3, 16]]",3
4,"[563, 360, 574, 373]",3,answer,"[{'box': [563, 360, 574, 373], 'text': '3'}]","[[15, 4]]",4


In [14]:
df_test.label.value_counts()

question    1077
answer       821
other        312
header       122
Name: label, dtype: int64

In [15]:
df_test.isna().sum()

box        0
text       0
label      0
words      0
linking    0
id         0
dtype: int64

In [16]:
text_list_train = df_train['text'].tolist()
text_list_test = df_test['text'].tolist()

In [17]:
df_train.shape, df_test.shape

((7411, 6), (2332, 6))

In [18]:
# Added extra code here - DO NOT RUN
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()

df_train['label'] = labelencoder.fit_transform(df_train['label'])
df_test['label'] = labelencoder.fit_transform(df_test['label'])

In [19]:
# Added extra code here - DO NOT RUN
df_test.label.value_counts()

3    1077
0     821
2     312
1     122
Name: label, dtype: int64

In [20]:
# Added extra code here - DO NOT RUN
df_train.label.value_counts()

3    3266
0    2802
2     902
1     441
Name: label, dtype: int64

In [21]:
# Setting up the device for GPU usage
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)

cpu


In [22]:
from transformers import BertTokenizer
import torch


tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# Defining some key variables that will be used later on in the training
MAX_LEN = 100
TRAIN_BATCH_SIZE = 16
VALID_BATCH_SIZE = TRAIN_BATCH_SIZE*2
# EPOCHS = 1
LEARNING_RATE = 2e-05
# Change the pre-trained bert model
#tokenizer = BertTokenizer.from_pretrained('roberta-base') #Cased 

In [23]:
class SentimentData(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe.text
        self.targets = self.data.label
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]

        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float),
        }

In [24]:
train_size = 1
train_data=df_train.sample(frac=train_size,random_state=200)
train_data = train_data.reset_index(drop=True)

test_size = 1
test_data=df_test.sample(frac=test_size,random_state=200)
test_data = test_data.reset_index(drop=True)


print("FULL Dataset: {}".format(df_train.shape))
print("TRAIN Dataset: {}".format(train_data.shape))
print("TEST Dataset: {}".format(test_data.shape))

training_set = SentimentData(train_data, tokenizer, MAX_LEN)
test_set = SentimentData(test_data, tokenizer, MAX_LEN)

FULL Dataset: (7411, 6)
TRAIN Dataset: (7411, 6)
TEST Dataset: (2332, 6)


In [25]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': False,
                'num_workers': 0
                }
training_loader = DataLoader(training_set, **train_params)


test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': False,
                'num_workers': 0
                }

testing_loader = DataLoader(test_set, **test_params)

In [26]:
class Feature_Extractor(torch.nn.Module):
    def __init__(self):
        super(Feature_Extractor, self).__init__()
        #bert-base-cased 768
        #bert-large-cased bert-large-uncased 1024
        #roberta-base-cased 768
        #biobert

        self.l1 = AutoModel.from_pretrained('bert-base-uncased')# BERT large
        self.pre_classifier = torch.nn.Linear(768, 768)

    def forward(self,input_ids,attention_mask,token_type_ids):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        
        return pooler

In [27]:
model = Feature_Extractor()
model.to(device)# Defining the training function on the 80% of the dataset for tuning the distilbert model

def extracing(data_loader):
    output = []
    model.eval()
    for _,data in tqdm(enumerate(data_loader, 0)):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.long)
        outputs = model(ids, mask,token_type_ids)
        output.extend(outputs.tolist())
    return output


In [28]:
train = extracing(training_loader)


0it [00:00, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/Users/shashanksahoo/miniconda3/envs/gpt-LLM/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:2137: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
464it [09:56,  1.29s/it]


In [30]:
test = extracing(testing_loader)

73it [03:12,  2.63s/it]


In [31]:
len(train), len(test)

(7411, 2332)

In [35]:
len(train[0]), len(test[0])

(768, 768)

In [38]:
torch.__version__

'2.0.0'

In [36]:
torch.save(model, '../models/funsd_bert_embedder_v3.pt')

In [37]:
model_scripted = torch.jit.script(model) # Export to TorchScript
model_scripted.save('../models/funsd_bert_embedder_v4.pt') # Save

UnsupportedNodeError: try blocks aren't supported:
  File "/Users/shashanksahoo/miniconda3/envs/gpt-LLM/lib/python3.9/site-packages/transformers/modeling_utils.py", line 148
        device).
        """
        try:
        ~~~ <--- HERE
            return next(self.parameters()).device
        except StopIteration:


In [39]:
torch.save(model.state_dict(), '../models/funsd_bert_embedder_v4.pt')

In [45]:
import pickle
with open('../funsd/textual_feature/funsd_train_bert_cls_emb.pkl','wb') as f:
  pickle.dump(train,f)

In [46]:
import pickle
with open('../funsd/textual_feature/funsd_test_bert_cls_emb.pkl','wb') as f:
  pickle.dump(test,f)

In [65]:
df_train.columns

Index(['box', 'text', 'label', 'words', 'linking', 'id'], dtype='object')

In [66]:
df_test.columns

Index(['box', 'text', 'label', 'words', 'linking', 'id'], dtype='object')

In [67]:
df_train['bert_large_emb'] = train

In [68]:
df_test['bert_large_emb'] = test

In [69]:
import json 
with open("../data/funsd_full_feature_sg_train_positional_1.json", "r") as fp:
    train_list_dict = json.load(fp)
    
with open("../data/funsd_full_feature_sg_test_positional_1.json", "r") as fp:
    eval_list_dict = json.load(fp)

In [70]:
train_list_dict.keys()

dict_keys(['92091873', '91939637', '87533049', '01073843', '92586242', '0012529284', '71341634', '0001477983', '91315069_91315070', '0060207528', '91161344_91161347', '71366499', '00922237', '91355841', '93380187', '01150773_01150774', '0060165115', '91914407', '92081358_1359', '660978', '0011974919', '81749056_9057', '00093726', '0000999294', '81186212', '0060077689', '87672097', '11875011', '00851772_1780', '0011859695', '81619511_9513', '0001456787', '81310636', '0060080406', '0011973451', '91104867', '00836244', '0001476912', '12052385', '0060255888', '71206427', '0012199830', '0001463448', '0000990274', '716552', '00920222', '0012529295', '80728670', '93213298', '71108371', '0001209043', '88057519', '01191071_1072', '00836816', '91361993', '92298125', '00040534', '0011838621', '0060029036', '0000989556', '0060024314', '0001129658', '0071032790', '00838511_00838525', '87682908', '0060214859', '00070353', '93351929_93351931', '0001438955', '81619486_9488', '00920294', '0000971160', 

In [71]:
for l in train_list_dict:
  for i, obj in enumerate(train_list_dict[l]['objects']):
    train_list_dict[l]['objects'][obj]['bert_large_emb'] = train[i]
    

In [72]:
for l in eval_list_dict:
  for i, obj in enumerate(eval_list_dict[l]['objects']):
    eval_list_dict[l]['objects'][obj]['bert_large_emb'] = test[i]
    

In [73]:
train_list_dict['92091873']['objects']['1']

{'id': 1,
 'box': [425, 130, 475, 145],
 'category': 'question',
 'text': 'Division',
 'relations': {'0': {'id': 70, 'object': 0},
  '1': {'id': 71, 'object': 2},
  '2': {'id': 72, 'object': 3},
  '3': {'id': 73, 'object': 4},
  '4': {'id': 74, 'object': 5},
  '5': {'id': 75, 'object': 6},
  '6': {'id': 76, 'object': 7},
  '7': {'id': 77, 'object': 8},
  '8': {'id': 78, 'object': 9},
  '9': {'id': 79, 'object': 10},
  '10': {'id': 80, 'object': 11},
  '11': {'id': 81, 'object': 12},
  '12': {'id': 82, 'object': 13},
  '13': {'id': 83, 'object': 14},
  '14': {'id': 84, 'object': 15},
  '15': {'id': 85, 'object': 16},
  '16': {'id': 86, 'object': 17},
  '17': {'id': 87, 'object': 18},
  '18': {'id': 88, 'object': 19},
  '19': {'id': 89, 'object': 20},
  '20': {'id': 90, 'object': 21},
  '21': {'id': 91, 'object': 22},
  '22': {'id': 92, 'object': 23},
  '23': {'id': 93, 'object': 24},
  '24': {'id': 94, 'object': 25},
  '25': {'id': 95, 'object': 26},
  '26': {'id': 96, 'object': 27},
  

In [74]:
import json 
with open("../data/funsd_full_feature_sg_train_positional_1_edited.json", "w") as fp:
    json.dump(train_list_dict, fp)
    
with open("../data/funsd_full_feature_sg_test_positional_1_edited.json", "w") as fp:
    json.dump(eval_list_dict, fp)